# DCA

In [1]:
import numpy as np
import cplex


k = 30
m = 50
R = np.random.uniform(low = 5,high=10,size=(k,m))
h = 3

E = np.sort(R,axis=0)
E = E[k-3:]
E = np.sum(E,axis=0)

def objective(E,X):
  return np.linalg.norm(E-X.dot(R))

In [2]:
def make_team_vector(indices):
  v = np.zeros(k)
  v[indices] = 1
  return v

### Giải bằng greedy tìm global

In [3]:
# brute force find global optima :
best_team = [0,1,2]
lowest_distance = 2e8
for first in range(k-2):
  for second in range(first,k-1):
    for third in range(second,k):
      team = np.zeros(k)
      team[[first,second,third]] = 1 # vector X_i [0,...1...1...1..0]
      d = objective(E,team) # tính khoảng cách 
      if( d < lowest_distance):
        best_team = [first,second,third]
        lowest_distance = d
        
print('best team is : ',best_team,' with distance = ',lowest_distance)

best team is :  [5, 12, 20]  with distance =  40.97753687084861


### Giải G(x) với solver :

Hệ số truyền vào cplex :
- quadratic matrix: 2 * R.dot(R.T) ( k * k )
- linear : -2 * (E.dot(R.T))

In [4]:
ub = [1.0] *k # bound của  0 <= x <=1 
lb = [0.0] *k 
A = 2 * R.dot(R.T)
names = ['x'+str(i) for i in range(k)]
linexp = cplex.SparsePair(ind=names,val=[1.0] *k)
row = [i for i in range(k)]
qmat = [[row,A[i]] for i in range(k)] # Hệ số A để truyền vào cplex 
# first_part = E.dot(R.T)  # Phần E.dot (R.T)


b = -2 * E.dot(R.T) # không gồm phần tau * xl 

p = cplex.Cplex() # make problem

p.objective.set_sense(p.objective.sense.minimize)

p.variables.add(obj=b,ub=ub,lb=lb,names=names,types=['I']*k) #  0 <= x_i <=1

p.linear_constraints.add(lin_expr=[linexp], 
                       rhs=[3.0],senses="E") # x0+x1 +... + xk = 3
p.objective.set_quadratic(qmat) # set hệ số cho A

p.solve()

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve added 870 rows and 435 columns.
Reduced MIP has 871 rows, 465 columns, and 1770 nonzeros.
Reduced MIP has 465 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.12 ticks)
Found incumbent of value -39498.406632 after 0.01 sec. (0.41 ticks)
Probing time = 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 435 rows and 0 columns.
Reduced MIP has 436 rows, 465 columns, and 1335 nonzeros.
Reduced MIP has 465 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.05 ticks)
Probing time = 0.00 sec. (0.09 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.46 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

In [5]:
np.array(p.solution.get_values())

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:
team = np.array(p.solution.get_values())
team = np.round(team).nonzero()
print(team)
print(objective(E,make_team_vector(team)))

(array([ 5, 12, 20], dtype=int64),)
40.97753687084861


### Giải bằng G(x) - h(x) với solver,  $X_i \in R$

In [7]:
def make_problem(new_b):
  p = cplex.Cplex() # make problem

  p.objective.set_sense(p.objective.sense.minimize)

  p.variables.add(obj=new_b,ub=ub,lb=lb,names=names) #  0 <= x_i <=1

  p.linear_constraints.add(lin_expr=[linexp], 
                       rhs=[3.0],senses="E") # x0+x1 +... + xk = 3
  p.objective.set_quadratic(qmat) # set hệ số cho A

  p.solve()
  return p

In [8]:
list_tau = [0.1,1,10,20,30,50,100,1000,10000]

In [9]:
first_part = E.dot(R.T)

In [ ]:
mat_result = {}
X_0 = np.zeros(k) 

for tau in list_tau:
  dis = []
  list_teams = []
  b_0 = E.dot(R.T) - tau *(2*X_0 - 1)
  p = make_problem(b_0)  
  p.solve()
  l = 0
  while True:
    result = p.solution.get_values()
    team = np.array(result)
    so = (-team).argsort()[:3]
    team = np.zeros(k)
    team[so] = 1
    list_teams.append(team.nonzero())
    dist = objective(E,team)
    dis.append(dist)
    if l > 1 :
      if(dist > dis[l-1]):
        print(str(l),'#######InCorrect######') # sai vì dist bắt buộc phải giảm 
        break
      else:
        if(dist == dis[l-1]):
          break # break khi distance không đổi  
    new_b = -2 * first_part - tau * (2*team - 1) # update b
    p = make_problem(new_b)
    p.solve()
    l+=1
  mat_result[tau] = list_teams

In [11]:
mat_result

{0.1: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 1: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 10: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 20: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 30: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 50: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 100: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 1000: [(array([10, 15, 16], dtype=int64),),
  (array([10, 15, 16], dtype=int64),),
  (array([10, 15, 16], dtype=int64),)],
 10000: [(array([10, 

### Giải G(x) - H(x), điều kiện $X_i \in N$

In [12]:
def make_problem2(new_b):
  p = cplex.Cplex() # make problem

  p.objective.set_sense(p.objective.sense.minimize)

  p.variables.add(obj=new_b,ub=ub,lb=lb,names=names,types=['I']*k) #  0 <= x_i <=1

  p.linear_constraints.add(lin_expr=[linexp], 
                       rhs=[3.0],senses="E") # x0+x1 +... + xk = 3
  p.objective.set_quadratic(qmat) # set hệ số cho A

  p.solve()
  return p

In [ ]:
mat_result = {}
X_0 = np.zeros(k) 

for tau in list_tau:
  dis = []
  list_teams = []
  b_0 = E.dot(R.T) - tau *(2*X_0 - 1)
  p = make_problem(b_0)  
  p.solve()
  l = 0
  while True:
    result = p.solution.get_values()
    team = np.array(result)
    so = (-team).argsort()[:3]
    team = np.zeros(k)
    team[so] = 1
    list_teams.append(team.nonzero())
    dist = objective(E,team)
    dis.append(dist)
    if l > 1 :
      if(dist > dis[l-1]):
        print(str(l),'#######InCorrect######') # sai vì dist bắt buộc phải giảm 
        break
      else:
        if(dist == dis[l-1]):
          break # break khi distance không đổi  
    new_b = -2 * first_part - tau * (2*team - 1) # update b
    p = make_problem2(new_b)
    p.solve()
    l+=1
  mat_result[tau] = list_teams

In [14]:
mat_result

{0.1: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 1: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 10: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 20: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 30: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 50: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 100: [(array([10, 15, 16], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),),
  (array([ 5, 12, 20], dtype=int64),)],
 1000: [(array([10, 15, 16], dtype=int64),),
  (array([10, 15, 16], dtype=int64),),
  (array([10, 15, 16], dtype=int64),)],
 10000: [(array([10, 

- Các case có nghiệm đầu giống nhau vì có cùng X_0 
- Khi $\tau$ quá lớn, DCA giải không ra nghiệm 